<a href="https://colab.research.google.com/github/AhmedRezaShams/Python/blob/main/4_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas scikit-learn nltk bnlp

In [ ]:
pip install bnlp_toolkit

In [ ]:
pip install -U bnlp_toolkit

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import nltk
from bnlp import NLTKTokenizer

In [ ]:
import pandas as pd

# Specify the file path of the XLSX file
file_path = "/content/Data Mining.xlsx"

# Read the XLSX file into a pandas DataFrame
try:
    data = pd.read_excel(file_path)
except UnicodeDecodeError:
    print("Error: Failed to decode XLSX file. Please check the encoding.")
except FileNotFoundError:
    print(f"Error: File '{file_path}' not found.")

# Now you can work with the 'data' DataFrame containing the contents of the XLSX file.


In [ ]:
class_column = 'class'
unique_classes = data[class_column].unique()
num_classes = len(unique_classes)

print(f"Number of unique classes: {num_classes}")
print("Unique classes:", unique_classes)

Number of unique classes: 2
Unique classes: [0 1]


In [ ]:
class_column = 'class'

# Count the occurrences of each class
class_counts = data[class_column].value_counts()

print("Class distribution:")
print(class_counts)

Class distribution:
0    267
1    267
Name: class, dtype: int64


In [ ]:
data

,content,class,Unnamed: 2
0,"In this paper, the design of 3D Kanji ...",0,Original=0
1,This paper introduces the concept of designing...,1,AI=1
2,"3D Kanji is a 3D form of a Kanji character, wh...",0,NaN
3,A multimedia dictionary using 3D Kanji f...,1,NaN
4,The impression of 3D Kanji can be changed by t...,0,NaN
...,...,...,...
529,I would like to express my gratitude for the i...,1,NaN
530,And the total number of pixels of part ...,0,NaN
531,The images taken on July 22 and August 8 exhib...,1,NaN
532,Model formula which finds predictions of ...,0,NaN


In [ ]:
pip install tensorflow keras scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.utils import to_categorical



# Split the dataset into features (X) and labels (y)
X = data['content']
y = data['class']

# Encode the target labels to numerical values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to make them of the same length
max_sequence_length = 500  # You can adjust this value based on your dataset and text length
X_train_padded = pad_sequences(X_train_seq, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_sequence_length)

# Convert the target labels to one-hot encoding for 2 classes
y_train_onehot = to_categorical(y_train, num_classes=2)
y_test_onehot = to_categorical(y_test, num_classes=2)


In [ ]:
# Create the LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length))
model.add(LSTM(units=128))
model.add(Dropout(0.2))
model.add(Dense(units=2, activation='relu'))  # For 3-class classification

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train_padded, y_train_onehot, batch_size=32, epochs=100, validation_data=(X_test_padded, y_test_onehot))

# Evaluate the model
_, accuracy = model.evaluate(X_test_padded, y_test_onehot)
print("Accuracy:", accuracy)


Epoch 1/100
14/14 [==============================] - 17s 946ms/step - loss: 1.7840 - accuracy: 0.4965 - val_loss: 0.7300 - val_accuracy: 0.4299
Epoch 2/100
14/14 [==============================] - 14s 941ms/step - loss: 0.8983 - accuracy: 0.5293 - val_loss: 1.0752 - val_accuracy: 0.4112
Epoch 3/100
14/14 [==============================] - 14s 992ms/step - loss: 1.0131 - accuracy: 0.6019 - val_loss: 1.0349 - val_accuracy: 0.3925
Epoch 4/100
14/14 [==============================] - 14s 1s/step - loss: 0.9143 - accuracy: 0.6019 - val_loss: 0.9203 - val_accuracy: 0.4019
Epoch 5/100
14/14 [==============================] - 14s 1s/step - loss: 0.7861 - accuracy: 0.5761 - val_loss: 0.7879 - val_accuracy: 0.4299
Epoch 6/100
14/14 [==============================] - 14s 1000ms/step - loss: 0.6891 - accuracy: 0.5293 - val_loss: 0.7500 - val_accuracy: 0.4299
Epoch 7/100
14/14 [==============================] - 15s 1s/step - loss: 0.6580 - accuracy: 0.5902 - val_loss: 0.7484 - val_accuracy: 0.3925


In [ ]:
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Get the model predictions on the test set
y_pred_prob = model.predict(X_test_padded)
y_pred_labels = np.argmax(y_pred_prob, axis=1)

# Invert one-hot encoding for y_test
y_test_labels = np.argmax(y_test_onehot, axis=1)

# Calculate F1 score
f1 = f1_score(y_test_labels, y_pred_labels, average='macro')
print("F1 Score:", f1)

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test_labels, y_pred_labels)
print("Confusion Matrix:")
print(conf_matrix)

# Calculate ROC-AUC curve
roc_auc = roc_auc_score(y_test_onehot, y_pred_prob, average='macro')
print("ROC-AUC Score:", roc_auc)

# Plot ROC curve for each class
fpr = dict()
tpr = dict()
roc_auc_per_class = dict()



4/4 [==============================] - 2s 407ms/step
F1 Score: 0.4013986013986014
Confusion Matrix:
[[23 38]
 [26 20]]
ROC-AUC Score: 0.36003207412687094


In [ ]:
# Make predictions on the user input
user_text = input("Enter text: ")
user_text_seq = tokenizer.texts_to_sequences([user_text])
user_text_padded = pad_sequences(user_text_seq, maxlen=max_sequence_length)
user_prediction = model.predict(user_text_padded)
user_prediction_label = np.argmax(user_prediction)  # Get the class index with the highest probability
user_prediction_class = label_encoder.inverse_transform([user_prediction_label])[0]

print("Predicted :", user_prediction_class)

Enter text: A multimedia dictionary using 3D Kanji has been  proposed. This dictionary uses a combination of 3D  Kanji  and  multimedia  name  to  provide  a  multiple  view  of  objects  and  phenomena.  The  3D  Kanji  represents  meaning  through  its  font,  weight,  and  structured  design.  The  multimedia  name  describes  meaning  through  a  set  of  structured  pictures.  An  example of applying the approach as a dictionary for Functions of two holes 
1/1 [==============================] - 0s 81ms/step
Predicted : 0


In [ ]:
pip install tensorflow tensorflow-datasets transformers

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.preprocessing import LabelEncoder



# Split the dataset into features (X) and labels (y)
X = data['content']
y = data['class']

# Use LabelEncoder to convert labels to integers
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Tokenize the text and convert them into input features
input_ids = []
attention_masks = []

for text in X:
    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='tf'
    )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# Convert lists to tensors
input_ids = tf.concat(input_ids, axis=0)
attention_masks = tf.concat(attention_masks, axis=0)

# Convert to NumPy arrays
X_np = np.array(input_ids)
y_np = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, test_size=0.2, random_state=42)

# Create TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# Batch the data and shuffle
batch_size = 32
train_dataset = train_dataset.shuffle(len(X_train)).batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

# Load pre-trained BERT model for sequence classification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

# Set up optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train the model with hyperparameter tuning
epochs = 6
history = model.fit(train_dataset, epochs=epochs, validation_data=test_dataset)

# Evaluation on test data
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy*100:.2f}%")


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/6
14/14 [==============================] - 796s 54s/step - loss: 1.3020 - accuracy: 0.4122 - val_loss: 0.9782 - val_accuracy: 0.4112
Epoch 2/6
14/14 [==============================] - 691s 49s/step - loss: 0.8925 - accuracy: 0.5269 - val_loss: 0.8229 - val_accuracy: 0.4206
Epoch 3/6
14/14 [==============================] - 726s 52s/step - loss: 0.7867 - accuracy: 0.5504 - val_loss: 0.7999 - val_accuracy: 0.4206
Epoch 4/6
14/14 [==============================] - 717s 51s/step - loss: 0.7173 - accuracy: 0.6066 - val_loss: 0.7784 - val_accuracy: 0.4206
Epoch 5/6
14/14 [==============================] - 715s 51s/step - loss: 0.6721 - accuracy: 0.6604 - val_loss: 0.7369 - val_accuracy: 0.5514
Epoch 6/6
4/4 [==============================] - 48s 11s/step - loss: 0.7592 - accuracy: 0.5047
Test Accuracy: 50.47%


In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
import numpy as np

In [ ]:
# Load the pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)  # Change num_labels based on your problem

class_names = ['1', '0']  # Replace with actual class names


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def predict_text(text):
    # Tokenize and preprocess the input text
    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='tf'
    )

    input_ids = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask']

    # Make a prediction using the model
    logits = model([input_ids, attention_mask])[0]
    predicted_class = tf.argmax(logits, axis=1).numpy()[0]
    prediction = class_names[predicted_class]

    return prediction

# Get user input and make predictions
user_input = input("Enter text: ")
prediction = predict_text(user_input)
print("Prediction:", prediction)

Enter text: A multimedia dictionary using 3D Kanji has been  proposed. This dictionary uses a combination of 3D  Kanji  and  multimedia  name  to  provide  a  multiple  view  of  objects  and  phenomena.  The  3D  Kanji  represents  meaning  through  its  font,  weight,  and  structured  design.  The  multimedia  name  describes  meaning  through  a  set  of  structured  pictures.  An  example of applying the approach as a dictionary for Functions of two holes 
Prediction: 1


In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.models import Model

import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.models import Model


In [ ]:
# Split the dataset into features (X) and labels (y)
X = data['content']
y = data['class']

# Use LabelEncoder to convert labels to integers
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)



# Define vocabulary size and embedding dimensions
vocab_size = tokenizer.vocab_size
embedding_dim = 100  # You can adjust this dimension based on your preference and dataset characteristics

# Input layer
input_layer = Input(shape=(128,))  # Assuming the maximum sequence length is 128

# Embedding layer
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(input_layer)

# Convolutional layers
conv_layer = Conv1D(filters=128, kernel_size=5, activation='relu')(embedding_layer)
global_max_pooling = GlobalMaxPooling1D()(conv_layer)

# Dense layers
dense_layer = Dense(512, activation='relu')(global_max_pooling)
output_layer = Dense(1, activation='sigmoid')(dense_layer)

# Create the model
cnn_model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
cnn_model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [ ]:
# Convert X_train to NumPy array if it's not already
X_train = np.array(input_ids)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size=0.9, random_state=42)

# Train the model
cnn_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluation on test data
loss, accuracy = cnn_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")


Epoch 1/100
2/2 [==============================] - 0s 172ms/step - loss: 0.2489 - accuracy: 0.6667 - val_loss: 0.1705 - val_accuracy: 0.7778
Epoch 2/100
2/2 [==============================] - 0s 149ms/step - loss: 0.0604 - accuracy: 0.9286 - val_loss: 0.1559 - val_accuracy: 0.7963
Epoch 3/100
2/2 [==============================] - 0s 154ms/step - loss: 0.0201 - accuracy: 0.9762 - val_loss: 0.1414 - val_accuracy: 0.8148
Epoch 4/100
2/2 [==============================] - 0s 166ms/step - loss: 2.8708e-04 - accuracy: 1.0000 - val_loss: 0.1406 - val_accuracy: 0.8148
Epoch 5/100
2/2 [==============================] - 0s 161ms/step - loss: 1.6700e-05 - accuracy: 1.0000 - val_loss: 0.1400 - val_accuracy: 0.8148
Epoch 6/100
2/2 [==============================] - 0s 161ms/step - loss: 4.3473e-06 - accuracy: 1.0000 - val_loss: 0.1392 - val_accuracy: 0.8148
Epoch 7/100
2/2 [==============================] - 0s 163ms/step - loss: 3.0334e-06 - accuracy: 1.0000 - val_loss: 0.1386 - val_accuracy: 0.83

In [ ]:
def predict_text_cnn(text):
    encoded_text = tokenizer.encode(text, add_special_tokens=True)
    padded_text = tf.keras.preprocessing.sequence.pad_sequences([encoded_text], maxlen=128, padding='post')

    prediction = cnn_model.predict(padded_text)
    predicted_class = 1 if prediction > 0.5 else 0
    prediction_label = class_names[predicted_class]

    return prediction_label


In [ ]:
# Get user input and make predictions
user_input = input("Enter text: ")
prediction = predict_text_cnn(user_input)
print("Prediction:", prediction)




In [ ]:

import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Input, Embedding, GRU, Dense
from tensorflow.keras.models import Model

In [ ]:
# Split the dataset into features (X) and labels (y)
X = data['content']
y = data['class']

# Use LabelEncoder to convert labels to integers
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Tokenize the text and convert them into input features
# ...

# Define vocabulary size and embedding dimensions
vocab_size = tokenizer.vocab_size
embedding_dim = 100  # You can adjust this dimension based on your preference and dataset characteristics

# Input layer
input_layer = Input(shape=(128,))  # Assuming the maximum sequence length is 128

# Embedding layer
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(input_layer)

# GRU layer
gru_layer = GRU(128)(embedding_layer)

# Dense layers
dense_layer = Dense(64, activation='relu')(gru_layer)
output_layer = Dense(1, activation='sigmoid')(dense_layer)

In [ ]:
# Create the model
gru_model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
gru_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Convert X_train to NumPy array if it's not already
X_train = np.array(input_ids)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size=0.9, random_state=42)

# Train the model
gru_model.fit(X_train, y_train, epochs=70, batch_size=32, validation_split=0.2)

# Evaluation on test data
loss, accuracy = gru_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")

Epoch 1/70
2/2 [==============================] - 2s 447ms/step - loss: 4.3325e-05 - accuracy: 1.0000 - val_loss: 3.6955 - val_accuracy: 0.1818
Epoch 2/70
2/2 [==============================] - 0s 113ms/step - loss: 2.4336e-06 - accuracy: 1.0000 - val_loss: 4.3523 - val_accuracy: 0.1818
Epoch 3/70
2/2 [==============================] - 0s 123ms/step - loss: 4.2977e-06 - accuracy: 1.0000 - val_loss: 4.8207 - val_accuracy: 0.2727
Epoch 4/70
2/2 [==============================] - 0s 107ms/step - loss: 6.2092e-06 - accuracy: 1.0000 - val_loss: 5.0981 - val_accuracy: 0.2727
Epoch 5/70
2/2 [==============================] - 0s 111ms/step - loss: 4.2851e-06 - accuracy: 1.0000 - val_loss: 5.2150 - val_accuracy: 0.2727
Epoch 6/70
2/2 [==============================] - 0s 107ms/step - loss: 2.6386e-06 - accuracy: 1.0000 - val_loss: 5.2384 - val_accuracy: 0.2727
Epoch 7/70
2/2 [==============================] - 0s 111ms/step - loss: 1.2951e-06 - accuracy: 1.0000 - val_loss: 5.2320 - val_accuracy:

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer  # You might need to import this depending on your tokenization
import numpy as np


In [ ]:
# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Load your GRU model here
# gru_model = ...

class_names = ['1', '0']  # Replace with actual class names

In [ ]:
def predict_text_gru(text):
    # Tokenize and preprocess the input text
    encoded_text = tokenizer.encode(text, add_special_tokens=True)
    padded_text = tf.keras.preprocessing.sequence.pad_sequences([encoded_text], maxlen=128, padding='post')

    # Make a prediction using the GRU model
    prediction = gru_model.predict(padded_text)
    predicted_class = 1 if prediction > 0.5 else 0
    prediction_label = class_names[predicted_class]

    return prediction_label

# Get user input and make predictions
user_input = input("Enter text: ")
prediction = predict_text_gru(user_input)
print("Prediction:", prediction)

Enter text: A multimedia dictionary using 3D Kanji has been  proposed. This dictionary uses a combination of 3D  Kanji  and  multimedia  name  to  provide  a  multiple  view  of  objects  and  phenomena.  The  3D  Kanji  represents  meaning  through  its  font,  weight,  and  structured  design.  The  multimedia  name  describes  meaning  through  a  set  of  structured  pictures.  An  example of applying the approach as a dictionary for Functions of two holes 
1/1 [==============================] - 0s 113ms/step
Prediction: 0
